# Import files

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import json
import os
from tqdm import tqdm_notebook 
import string
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [2]:
def file(path):
    fds = sorted(os.listdir(path))
    filenames = []
    for file in fds:
        if file.endswith(('.json')): 
            filenames.append(file)
    return filenames


In [3]:
columns=["contract_id","branch","branch_code","cred_line_id","depart_code","depart_name","dog_summa",
         "dog_summa_nt","grace_period","group_conv_num","kind_credit","method_calc_prc","name_group_client",
         "pod_sector_cred","prc_rate","pre_payment_acc","product","rate_admin_prc","sector_cred",
         "code_group_client","contract_manager_dep_code","stupen_cred","sum_admin_prc",
        "sum_admin_prc_nt","sum_cred_line","valuta","date_begin","date_end","date_open"]

## Load dataset for credit

In [4]:
#Read json files and write credit and client_id in new dataframe
path = '/home/student/Desktop/cs2018-07-17'
filenames = file(path)
data=pd.DataFrame(columns=columns)
for file in tqdm_notebook(filenames):
    f = open(path + '/' + file)
    my_dict = json.load(f)
    f.close()
    for i in range (len(my_dict["credit"])):
        row=my_dict["credit"][i]
        row["client_id"]=my_dict["client_id"]
        data=data.append(row,ignore_index=True)

In [5]:
#Save in csv format
data.to_csv("credit_from_json.csv",index=False)

# Credit data preprocessing

In [6]:
#Drop information about collaterals and date_open (duplicates date_begin)
data.drop(["collateral","date_open"],axis=1,inplace=True)

In [7]:
# Drop categorical values and values in international currency
data.drop(["branch","depart_name","cred_line_id","product","rate_admin_prc","sum_cred_line","dog_summa","sum_admin_prc"],axis=1,inplace=True)

In [8]:
data.columns

Index(['contract_id', 'branch_code', 'depart_code', 'dog_summa_nt',
       'grace_period', 'group_conv_num', 'kind_credit', 'method_calc_prc',
       'name_group_client', 'pod_sector_cred', 'prc_rate', 'pre_payment_acc',
       'sector_cred', 'code_group_client', 'contract_manager_dep_code',
       'stupen_cred', 'sum_admin_prc_nt', 'valuta', 'date_begin', 'date_end',
       'client_id', 'overdue'],
      dtype='object')

In [9]:
#New columns grooup_credit
data["group_credit"]=data.group_conv_num
data.drop(["group_conv_num","code_group_client","name_group_client"],axis=1,inplace=True)

In [10]:
index=data[data.group_credit.isnull()].index
data.group_credit.iloc[index]=0

/home/student/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
#If it is group credit than give 1, else 0
data.group_credit=data["group_credit"].apply(lambda x: 1 if x!=0 else 0)

In [12]:
#Count Kind of credit and group them(if less than 100)
a = data.kind_credit.value_counts()
m = data.kind_credit.isin(a.index[a<100])
data.loc[m, 'kind_credit'] = "Другие"
data.kind_credit.value_counts()

Бизнес-микро                                           20677
Агро-микро                                             13790
Истеъмоли (Нав)                                         8060
Умуми                                                   1531
Другие                                                   525
ABCI/Чорводори                                           187
Махсулоти карзи бо амонати сугуртави ва курби собит      130
Гурухи-оилави                                            123
Айёми дилхох-1                                           120
Бовари                                                   109
Name: kind_credit, dtype: int64

In [13]:
data.method_calc_prc.value_counts()

Аннуитетный        39107
Комбинированный     6142
Простой                3
Name: method_calc_prc, dtype: int64

In [14]:
#Categorical to binary
a=pd.get_dummies(data.method_calc_prc)
data=pd.concat([data,a],axis=1)

In [15]:
index=data.overdue.dropna().index
index

Int64Index([    3,     4,     5,     6,     7,     8,     9,    10,    11,
               12,
            ...
            45242, 45243, 45244, 45245, 45246, 45247, 45248, 45249, 45250,
            45251],
           dtype='int64', length=45147)

In [16]:
# Drop samples with no infomation about overdue
data=data.iloc[index]
data

,contract_id,branch_code,depart_code,dog_summa_nt,grace_period,kind_credit,method_calc_prc,pod_sector_cred,prc_rate,pre_payment_acc,...,sum_admin_prc_nt,valuta,date_begin,date_end,client_id,overdue,group_credit,Аннуитетный,Комбинированный,Простой
3,1637320955,001-14,001-14,57100.00,0,Умуми,Комбинированный,Хариду фуруш,42.6,26223972810131098001,...,0.00,TJS,2016-04-16,2018-09-16,1225971105,"[{'no': 1672, 'active_summa': 18323.31, 'activ...",0,0,1,0
4,3160568643,001-04,001-04,5000.00,0,Агро-микро,Аннуитетный,Истеъмолот,42.0,26223972910113342002,...,50.00,TJS,2017-12-29,2019-12-29,886064014,"[{'no': 5867, 'active_summa': 4753.09, 'active...",1,1,0,0
5,3393025348,001-18,001-18-01,5000.00,0,Агро-микро,Аннуитетный,Растанипарвари,42.0,26223972710143595003,...,50.00,TJS,2018-03-23,2019-03-25,1551551986,"[{'no': 163374, 'active_summa': 5000.0, 'activ...",0,1,0,0
6,2593810969,001-21,001-21,8000.00,0,Бизнес-микро,Аннуитетный,Истеъмолот,46.0,26223972810118493001,...,10.00,TJS,2017-06-05,2019-06-06,349594513,"[{'no': 16743, 'active_summa': 5866.15, 'activ...",0,1,0,0
7,3233769171,001-21,001-21,2000.00,0,Бизнес-микро,Аннуитетный,Истеъмолот,42.0,26223972110209086001,...,20.00,TJS,2018-01-26,2019-01-26,3230654910,"[{'no': 16746, 'active_summa': 1888.67, 'activ...",0,1,0,0
8,2924160024,001-14,001-14-02,8000.00,0,Бизнес-микро,Аннуитетный,Истеъмолот,42.0,26223972510141304001,...,80.00,TJS,2017-10-04,2019-04-04,1382875420,"[{'no': 16748, 'active_summa': 6282.48, 'activ...",0,1,0,0
9,3416183132,001-18,001-18-01,18000.00,0,Агро-микро,Аннуитетный,Растанипарвари,27.0,26223972810219499001,...,10.00,TJS,2018-04-02,2020-04-02,3406731174,"[{'no': 163132, 'active_summa': 18000.0, 'acti...",0,1,0,0
10,3421812332,001-03,001-03,54660.44,0,Бизнес-микро,Аннуитетный,Истеъмолот,24.0,26227840310130973001,...,9.97,USD,2018-04-05,2020-04-05,348004847,"[{'no': 163462, 'active_summa': 6200.0, 'activ...",0,1,0,0
11,2666940429,001-19,001-19,5000.00,0,Бизнес-микро,Аннуитетный,Истеъмолот,47.0,26223972610182056001,...,10.00,TJS,2017-07-02,2019-07-03,2664931199,"[{'no': 16749, 'active_summa': 3873.94, 'activ...",0,1,0,0
12,3063966959,001-21,001-21,2000.00,0,Истеъмоли (Нав),Аннуитетный,Истеъмолот,42.0,26223972810200724002,...,20.00,TJS,2017-11-25,2018-11-25,3060553609,"[{'no': 16750, 'active_summa': 1606.26, 'activ...",1,1,0,0


In [17]:
a=pd.get_dummies(data.pod_sector_cred)
data=pd.concat([data,a],axis=1)
data.head()

,contract_id,branch_code,depart_code,dog_summa_nt,grace_period,kind_credit,method_calc_prc,pod_sector_cred,prc_rate,pre_payment_acc,...,Занбурпарвари,Ипотека,Истехсолот,Истеъмоли,Истеъмолот,Мохипарвари,Растанипарвари,Хариду фуруш,Хизматрасони,Чорвопарвари
3,1637320955,001-14,001-14,57100.0,0,Умуми,Комбинированный,Хариду фуруш,42.6,26223972810131098001,...,0,0,0,0,0,0,0,1,0,0
4,3160568643,001-04,001-04,5000.0,0,Агро-микро,Аннуитетный,Истеъмолот,42.0,26223972910113342002,...,0,0,0,0,1,0,0,0,0,0
5,3393025348,001-18,001-18-01,5000.0,0,Агро-микро,Аннуитетный,Растанипарвари,42.0,26223972710143595003,...,0,0,0,0,0,0,1,0,0,0
6,2593810969,001-21,001-21,8000.0,0,Бизнес-микро,Аннуитетный,Истеъмолот,46.0,26223972810118493001,...,0,0,0,0,1,0,0,0,0,0
7,3233769171,001-21,001-21,2000.0,0,Бизнес-микро,Аннуитетный,Истеъмолот,42.0,26223972110209086001,...,0,0,0,0,1,0,0,0,0,0


In [18]:
a=pd.get_dummies(data.valuta)
data=pd.concat([data,a],axis=1)
data.head()

,contract_id,branch_code,depart_code,dog_summa_nt,grace_period,kind_credit,method_calc_prc,pod_sector_cred,prc_rate,pre_payment_acc,...,Истеъмоли,Истеъмолот,Мохипарвари,Растанипарвари,Хариду фуруш,Хизматрасони,Чорвопарвари,RUB,TJS,USD
3,1637320955,001-14,001-14,57100.0,0,Умуми,Комбинированный,Хариду фуруш,42.6,26223972810131098001,...,0,0,0,0,1,0,0,0,1,0
4,3160568643,001-04,001-04,5000.0,0,Агро-микро,Аннуитетный,Истеъмолот,42.0,26223972910113342002,...,0,1,0,0,0,0,0,0,1,0
5,3393025348,001-18,001-18-01,5000.0,0,Агро-микро,Аннуитетный,Растанипарвари,42.0,26223972710143595003,...,0,0,0,1,0,0,0,0,1,0
6,2593810969,001-21,001-21,8000.0,0,Бизнес-микро,Аннуитетный,Истеъмолот,46.0,26223972810118493001,...,0,1,0,0,0,0,0,0,1,0
7,3233769171,001-21,001-21,2000.0,0,Бизнес-микро,Аннуитетный,Истеъмолот,42.0,26223972110209086001,...,0,1,0,0,0,0,0,0,1,0


In [19]:
data.drop("valuta",axis=1,inplace=True)

In [20]:
data.shape

(45147, 36)

In [21]:
sum(data.pre_payment_acc.isnull())

0

In [23]:
data.method_calc_prc.value_counts()

Аннуитетный        39042
Комбинированный     6102
Простой                3
Name: method_calc_prc, dtype: int64

In [24]:
data.pod_sector_cred.value_counts()

Истеъмолот        25207
Чорвопарвари       6189
Растанипарвари     5142
Хизматрасони       3900
Хариду фуруш       3710
Истехсолот          798
Богпарвари          151
Занбурпарвари        23
Ипотека              12
Мохипарвари           9
Истеъмоли             2
Name: pod_sector_cred, dtype: int64

In [25]:
data.date_end.value_counts()

2019-03-11    356
2019-03-18    311
2019-03-25    304
2019-02-21    279
2019-04-20    264
2019-04-18    259
2019-04-07    257
2019-02-28    255
2019-01-24    254
2018-12-12    252
2018-11-01    251
2019-03-04    244
2019-01-10    242
2019-04-14    240
2019-02-14    230
2019-01-17    227
2018-10-17    227
2019-04-11    222
2019-01-31    220
2019-04-06    219
2018-12-26    218
2019-04-15    216
2019-03-28    214
2019-10-16    207
2018-11-15    199
2019-03-22    198
2019-03-21    197
2019-02-07    193
2019-04-13    188
2018-11-08    187
             ... 
2016-08-13      1
2021-12-16      1
2017-09-25      1
2020-05-30      1
2016-08-06      1
2020-05-16      1
2017-10-28      1
2016-07-24      1
2017-01-21      1
2015-10-01      1
2017-08-01      1
2017-10-29      1
2016-05-15      1
2018-02-24      1
2016-12-25      1
2017-05-12      1
2017-01-29      1
2015-11-15      1
2016-10-01      1
2016-08-28      1
2016-09-05      1
2015-11-23      1
2016-03-03      1
2020-09-07      1
2017-08-03

In [29]:
data.sector_cred.value_counts()

Истеъмолот        25219
Хочагии кишлок    11507
Сохибкори          8392
СОХИБКОРИ            16
ХОЧАГИИ КИШЛОК        7
ИСТЕЪМОЛОТ            2
Name: sector_cred, dtype: int64

In [30]:
data.pod_sector_cred.value_counts()

Истеъмолот        25207
Чорвопарвари       6189
Растанипарвари     5142
Хизматрасони       3900
Хариду фуруш       3710
Истехсолот          798
Богпарвари          151
Занбурпарвари        23
Ипотека              12
Мохипарвари           9
Истеъмоли             2
Name: pod_sector_cred, dtype: int64

In [31]:
data.columns

Index(['contract_id', 'branch_code', 'depart_code', 'dog_summa_nt',
       'grace_period', 'kind_credit', 'method_calc_prc', 'pod_sector_cred',
       'prc_rate', 'pre_payment_acc', 'sector_cred',
       'contract_manager_dep_code', 'stupen_cred', 'sum_admin_prc_nt',
       'date_begin', 'date_end', 'client_id', 'overdue', 'group_credit',
       'Аннуитетный', 'Комбинированный', 'Простой', 'Богпарвари',
       'Занбурпарвари', 'Ипотека', 'Истехсолот', 'Истеъмоли', 'Истеъмолот',
       'Мохипарвари', 'Растанипарвари', 'Хариду фуруш', 'Хизматрасони',
       'Чорвопарвари', 'RUB', 'TJS', 'USD'],
      dtype='object')

In [32]:
data.drop("method_calc_prc",axis=1,inplace=True) 
# Drop this column because new columns were generated based on its values

In [33]:
data.drop("pod_sector_cred",axis=1,inplace=True)
# Drop this column because new columns were generated based on its values

In [34]:
a=pd.get_dummies(data.kind_credit,prefix="kind of credit")
data=pd.concat([data,a],axis=1)
data.head()

,contract_id,branch_code,depart_code,dog_summa_nt,grace_period,kind_credit,prc_rate,pre_payment_acc,sector_cred,contract_manager_dep_code,...,kind of credit_ABCI/Чорводори,kind of credit_Агро-микро,kind of credit_Айёми дилхох-1,kind of credit_Бизнес-микро,kind of credit_Бовари,kind of credit_Гурухи-оилави,kind of credit_Другие,kind of credit_Истеъмоли (Нав),kind of credit_Махсулоти карзи бо амонати сугуртави ва курби собит,kind of credit_Умуми
3,1637320955,001-14,001-14,57100.0,0,Умуми,42.6,26223972810131098001,Сохибкори,001-14,...,0,0,0,0,0,0,0,0,0,1
4,3160568643,001-04,001-04,5000.0,0,Агро-микро,42.0,26223972910113342002,Истеъмолот,001-04,...,0,1,0,0,0,0,0,0,0,0
5,3393025348,001-18,001-18-01,5000.0,0,Агро-микро,42.0,26223972710143595003,Хочагии кишлок,None,...,0,1,0,0,0,0,0,0,0,0
6,2593810969,001-21,001-21,8000.0,0,Бизнес-микро,46.0,26223972810118493001,Истеъмолот,001-21,...,0,0,0,1,0,0,0,0,0,0
7,3233769171,001-21,001-21,2000.0,0,Бизнес-микро,42.0,26223972110209086001,Истеъмолот,001-21,...,0,0,0,1,0,0,0,0,0,0


In [35]:
data.drop("kind_credit",axis=1,inplace=True)
# Drop this column because new columns were generated based on its values

In [36]:
data.columns

Index(['contract_id', 'branch_code', 'depart_code', 'dog_summa_nt',
       'grace_period', 'prc_rate', 'pre_payment_acc', 'sector_cred',
       'contract_manager_dep_code', 'stupen_cred', 'sum_admin_prc_nt',
       'date_begin', 'date_end', 'client_id', 'overdue', 'group_credit',
       'Аннуитетный', 'Комбинированный', 'Простой', 'Богпарвари',
       'Занбурпарвари', 'Ипотека', 'Истехсолот', 'Истеъмоли', 'Истеъмолот',
       'Мохипарвари', 'Растанипарвари', 'Хариду фуруш', 'Хизматрасони',
       'Чорвопарвари', 'RUB', 'TJS', 'USD', 'kind of credit_ABCI/Чорводори',
       'kind of credit_Агро-микро', 'kind of credit_Айёми дилхох-1',
       'kind of credit_Бизнес-микро', 'kind of credit_Бовари',
       'kind of credit_Гурухи-оилави', 'kind of credit_Другие',
       'kind of credit_Истеъмоли (Нав)',
       'kind of credit_Махсулоти карзи бо амонати сугуртави ва курби собит',
       'kind of credit_Умуми'],
      dtype='object')

In [37]:
data.sector_cred=data.sector_cred.apply(lambda x: str(x).upper())

In [38]:
data.sector_cred.value_counts()

ИСТЕЪМОЛОТ        25221
ХОЧАГИИ КИШЛОК    11514
СОХИБКОРИ          8408
NONE                  4
Name: sector_cred, dtype: int64

In [39]:
a=pd.get_dummies(data.sector_cred,prefix="sector")
data=pd.concat([data,a],axis=1)
data.head()

,contract_id,branch_code,depart_code,dog_summa_nt,grace_period,prc_rate,pre_payment_acc,sector_cred,contract_manager_dep_code,stupen_cred,...,kind of credit_Бовари,kind of credit_Гурухи-оилави,kind of credit_Другие,kind of credit_Истеъмоли (Нав),kind of credit_Махсулоти карзи бо амонати сугуртави ва курби собит,kind of credit_Умуми,sector_NONE,sector_ИСТЕЪМОЛОТ,sector_СОХИБКОРИ,sector_ХОЧАГИИ КИШЛОК
3,1637320955,001-14,001-14,57100.0,0,42.6,26223972810131098001,СОХИБКОРИ,001-14,1,...,0,0,0,0,0,1,0,0,1,0
4,3160568643,001-04,001-04,5000.0,0,42.0,26223972910113342002,ИСТЕЪМОЛОТ,001-04,4,...,0,0,0,0,0,0,0,1,0,0
5,3393025348,001-18,001-18-01,5000.0,0,42.0,26223972710143595003,ХОЧАГИИ КИШЛОК,None,3,...,0,0,0,0,0,0,0,0,0,1
6,2593810969,001-21,001-21,8000.0,0,46.0,26223972810118493001,ИСТЕЪМОЛОТ,001-21,6,...,0,0,0,0,0,0,0,1,0,0
7,3233769171,001-21,001-21,2000.0,0,42.0,26223972110209086001,ИСТЕЪМОЛОТ,001-21,1,...,0,0,0,0,0,0,0,1,0,0


In [40]:
data.drop("sector_cred",axis=1,inplace=True)

In [41]:
data.stupen_cred.value_counts()

1     19425
2     10543
3      6188
4      3595
5      2152
6      1314
7       706
8       459
9       288
10      148
11       95
12       55
14       43
13       37
15       24
16       18
17       16
18        9
20        7
19        6
25        6
21        4
29        4
26        2
22        2
23        1
Name: stupen_cred, dtype: int64

In [50]:
data.to_csv("credit_wt_categorigal.csv",index=False)

In [51]:
data=pd.read_csv("credit_wt_categorigal.csv")

In [42]:
#Pre_payment account info
# account contains 20 digits, where starting from 5th (5,6,7) digit there is currency info. Here we have accounts with three currencies (valuta)

#972 TJS
#840 USD
#810 RUB
code=data.pre_payment_acc.apply(lambda x: str(x))
data.pre_payment_acc=code.apply(lambda x: x[5:8])
data.pre_payment_acc=data.pre_payment_acc.apply(lambda x: "TJS" if x=="972" else "USD" if x=="840" else "RUB")
a=pd.get_dummies(data.pre_payment_acc,prefix="Prepayment account")
data=pd.concat([data,a],axis=1)
data.drop("pre_payment_acc",axis=1,inplace=True)


In [43]:
data.contract_manager_dep_code=data.contract_manager_dep_code.replace(np.NaN,0)
sum(data.contract_manager_dep_code.isna())

0

In [44]:
#drop pod-sector info
data.drop(['Богпарвари', 'Занбурпарвари', 'Ипотека', 'Истехсолот', 'Истеъмоли',
       'Истеъмолот', 'Мохипарвари', 'Растанипарвари', 'Хариду фуруш',
       'Хизматрасони', 'Чорвопарвари'],axis=1,inplace=True)

In [45]:
data.rename(columns={'kind of credit_ABCI/Чорводори':'kind of credit livestock', 'kind of credit_Агро-микро':'kind of credit micro_agro',
       'kind of credit_Айёми дилхох-1':'kind of credit any_day','kind of credit_Бизнес-микро':'kind of credit micro-business',
       'kind of credit_Бовари':'kind of credit chemicals', 'kind of credit_Гурухи-оилави':'kind of credit family_group',
       'kind of credit_Другие':'kind of credit Others', 'kind of credit_Истеъмоли (Нав)':'kind of credit new consumer',
        'kind of credit_Махсулоти карзи бо амонати сугуртави ва курби собит':'kind of credit credit_card','kind of credit_Умуми':'kind of credit general',
        'sector_NAN':'sector not given', 'sector_ИСТЕЪМОЛОТ':'sector consumer','sector_СОХИБКОРИ':'sector business',
        'sector_ХОЧАГИИ КИШЛОК':'sector housing'},inplace=True)

In [46]:
data.drop("overdue",axis=1,inplace=True)

In [47]:
data

,contract_id,branch_code,depart_code,dog_summa_nt,grace_period,prc_rate,contract_manager_dep_code,stupen_cred,sum_admin_prc_nt,date_begin,...,kind of credit new consumer,kind of credit credit_card,kind of credit general,sector_NONE,sector consumer,sector business,sector housing,Prepayment account_RUB,Prepayment account_TJS,Prepayment account_USD
3,1637320955,001-14,001-14,57100.00,0,42.6,001-14,1,0.00,2016-04-16,...,0,0,1,0,0,1,0,0,1,0
4,3160568643,001-04,001-04,5000.00,0,42.0,001-04,4,50.00,2017-12-29,...,0,0,0,0,1,0,0,0,1,0
5,3393025348,001-18,001-18-01,5000.00,0,42.0,0,3,50.00,2018-03-23,...,0,0,0,0,0,0,1,0,1,0
6,2593810969,001-21,001-21,8000.00,0,46.0,001-21,6,10.00,2017-06-05,...,0,0,0,0,1,0,0,0,1,0
7,3233769171,001-21,001-21,2000.00,0,42.0,001-21,1,20.00,2018-01-26,...,0,0,0,0,1,0,0,0,1,0
8,2924160024,001-14,001-14-02,8000.00,0,42.0,001-14-02,2,80.00,2017-10-04,...,0,0,0,0,1,0,0,0,1,0
9,3416183132,001-18,001-18-01,18000.00,0,27.0,001-18-01,1,10.00,2018-04-02,...,0,0,0,0,0,0,1,0,1,0
10,3421812332,001-03,001-03,54660.44,0,24.0,001-03,7,9.97,2018-04-05,...,0,0,0,0,1,0,0,0,0,1
11,2666940429,001-19,001-19,5000.00,0,47.0,001-19,1,10.00,2017-07-02,...,0,0,0,0,1,0,0,0,1,0
12,3063966959,001-21,001-21,2000.00,0,42.0,001-21,1,20.00,2017-11-25,...,1,0,0,0,1,0,0,0,1,0


In [48]:
data.to_csv("credit_cleaned.csv",index=False)